In [1]:
import cv2
import os

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Redimensionar para 224x224
    image = image / 255.0  # Normalizar para o intervalo [0, 1]
    return image

images = []
labels = []
for label in ["direita", "esquerda", "frente"]:
    for image_path in os.listdir(f"{label}"):
        image = preprocess_image(f"{label}/{image_path}")
        images.append(image)
        labels.append(label)

In [2]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Redimensionar para 224x224
    image = image / 255.0  # Normalizar para o intervalo [0, 1]
    return image

images = []
labels = []
label_to_index = {"direita": 0, "esquerda": 1, "frente": 2}

for label in ["direita", "esquerda", "frente"]:
    for image_path in os.listdir(f"{label}"):
        if image_path.endswith('.jpg') or image_path.endswith('.png'):  # Adicione qualquer outra extensão de imagem que você esteja usando
            image = preprocess_image(f"{label}/{image_path}")
            images.append(image)
            labels.append(label_to_index[label])  # Use o dicionário para converter o rótulo para um inteiro

# Converter listas para arrays numpy
images = np.array(images)
labels = np.array(labels)

# Codificar os rótulos como categorias
labels = to_categorical(labels, num_classes=3)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Construir o modelo
model = Sequential([
    Input(shape=(224, 224, 3)),  # Definir a forma da entrada aqui
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: direita, esquerda, frente
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))



Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 30s 312ms/step - accuracy: 0.6750 - loss: 1.3208 - val_accuracy: 0.7680 - val_loss: 0.5329
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 28s 309ms/step - accuracy: 0.8472 - loss: 0.3953 - val_accuracy: 0.7610 - val_loss: 0.5699
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 28s 306ms/step - accuracy: 0.8738 - loss: 0.3082 - val_accuracy: 0.7569 - val_loss: 0.6078
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 29s 314ms/step - accuracy: 0.9260 - loss: 0.2053 - val_accuracy: 0.7569 - val_loss: 0.8532
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 28s 312ms/step - accuracy: 0.9614 - loss: 0.1157 - val_accuracy: 0.7403 - val_loss: 0.9167
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 28s 306ms/step - accuracy: 0.9708 - loss: 0.0956 - val_accuracy: 0.7541 - val_loss: 0.9579
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 27s 300ms/step - accuracy: 0.9876 - loss: 0.0448 - val_accuracy: 0.7500 - val_loss: 1.1625
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 28s 308ms/step - accuracy: 0.9846 - loss: 0.0488 - val_accu

In [4]:
# Salvar o modelo no formato Keras
model.save('modelo_direcao.keras')